# Chapter 7: Surface Creation and Slab Models

## Learning Objectives
- Cut crystal surfaces using Miller indices
- Build slab models for surface calculations
- Understand surface terminations and reconstructions
- Add vacuum layers and adsorbates

---

## 7.1 What is a Surface Slab?

In computational materials science, we model surfaces using **slab models**:
- A slab is a finite-thickness piece of crystal
- Periodic in two dimensions (surface plane)
- Finite in the third dimension (surface normal)
- **Vacuum layer** separates periodic images

### Key Considerations:
1. **Surface orientation** (Miller indices)
2. **Slab thickness** (number of layers)
3. **Vacuum thickness** (to decouple periodic images)
4. **Surface termination** (which atoms are exposed)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple, Optional
from dataclasses import dataclass
import math

@dataclass
class LatticeParameters:
    a: float
    b: float
    c: float
    alpha: float = 90.0
    beta: float = 90.0
    gamma: float = 90.0

def lattice_vectors_from_parameters(params: LatticeParameters) -> np.ndarray:
    alpha = np.radians(params.alpha)
    beta = np.radians(params.beta)
    gamma = np.radians(params.gamma)
    
    a_vec = np.array([params.a, 0, 0])
    bx = params.b * np.cos(gamma)
    by = params.b * np.sin(gamma)
    b_vec = np.array([bx, by, 0])
    
    cx = params.c * np.cos(beta)
    cy = params.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma)
    cz = np.sqrt(max(0, params.c**2 - cx**2 - cy**2))
    c_vec = np.array([cx, cy, cz])
    
    return np.array([a_vec, b_vec, c_vec])

class Crystal:
    """Crystal structure with unit cell and atomic basis."""
    ELEMENTS = {
        'H':  {'Z': 1,  'color': 'white', 'radius': 0.31},
        'C':  {'Z': 6,  'color': 'gray', 'radius': 0.76},
        'N':  {'Z': 7,  'color': 'blue', 'radius': 0.71},
        'O':  {'Z': 8,  'color': 'red', 'radius': 0.66},
        'Na': {'Z': 11, 'color': 'violet', 'radius': 1.66},
        'Al': {'Z': 13, 'color': 'lightgray', 'radius': 1.21},
        'Si': {'Z': 14, 'color': 'gold', 'radius': 1.11},
        'Cu': {'Z': 29, 'color': 'brown', 'radius': 1.40},
        'Au': {'Z': 79, 'color': 'gold', 'radius': 1.36},
        'Pt': {'Z': 78, 'color': 'silver', 'radius': 1.39},
    }
    
    def __init__(self, name: str, lattice_vectors: np.ndarray):
        self.name = name
        self.lattice_vectors = np.array(lattice_vectors, dtype=float)
        self._inv_lattice = np.linalg.inv(self.lattice_vectors)
        self.basis: List[Tuple[str, np.ndarray]] = []
    
    @classmethod
    def from_parameters(cls, name: str, params: LatticeParameters) -> 'Crystal':
        vectors = lattice_vectors_from_parameters(params)
        return cls(name, vectors)
    
    def add_atom(self, symbol: str, fractional_coords: np.ndarray) -> None:
        self.basis.append((symbol, np.array(fractional_coords, dtype=float)))
    
    def add_atoms(self, symbol: str, positions: List[np.ndarray]) -> None:
        for pos in positions:
            self.add_atom(symbol, pos)
    
    @property
    def volume(self) -> float:
        return abs(np.linalg.det(self.lattice_vectors))
    
    @property
    def n_atoms(self) -> int:
        return len(self.basis)
    
    def fractional_to_cartesian(self, frac: np.ndarray) -> np.ndarray:
        return frac @ self.lattice_vectors
    
    def cartesian_to_fractional(self, cart: np.ndarray) -> np.ndarray:
        return cart @ self._inv_lattice
    
    def get_cartesian_positions(self) -> Tuple[List[str], np.ndarray]:
        symbols = [s for s, _ in self.basis]
        frac_positions = np.array([p for _, p in self.basis])
        cart_positions = self.fractional_to_cartesian(frac_positions)
        return symbols, cart_positions
    
    def make_supercell(self, na: int, nb: int, nc: int) -> 'Crystal':
        new_vectors = self.lattice_vectors * np.array([[na], [nb], [nc]])
        supercell = Crystal(f"{self.name}_{na}x{nb}x{nc}", new_vectors)
        
        for symbol, frac in self.basis:
            for i in range(na):
                for j in range(nb):
                    for k in range(nc):
                        new_frac = (frac + np.array([i, j, k])) / np.array([na, nb, nc])
                        supercell.add_atom(symbol, new_frac)
        return supercell

# Build FCC copper for examples
def build_fcc(element: str, a: float) -> Crystal:
    crystal = Crystal.from_parameters(f"{element}_FCC",
                                       LatticeParameters(a, a, a, 90, 90, 90))
    fcc_positions = [
        [0, 0, 0],
        [0.5, 0.5, 0],
        [0.5, 0, 0.5],
        [0, 0.5, 0.5]
    ]
    crystal.add_atoms(element, fcc_positions)
    return crystal

cu_bulk = build_fcc('Cu', 3.615)
print(f"Bulk copper: {cu_bulk}")

## 7.2 The Mathematics of Surface Cutting

To cut a surface with Miller indices (hkl), we need to:

1. **Find the surface normal** (perpendicular to the surface)
2. **Find two in-plane vectors** (periodic directions on the surface)
3. **Transform the crystal** so the surface normal becomes the z-axis

### Finding Surface Vectors

For cubic crystals, we can find in-plane vectors using:
- Two vectors perpendicular to [hkl]
- These define the surface unit cell

In [ ]:
def find_surface_vectors(h: int, k: int, l: int, 
                          lattice_vectors: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Find vectors defining a surface slab.
    
    Returns:
        (surface_normal, in_plane_v1, in_plane_v2)
        All in Cartesian coordinates
    """
    # Calculate reciprocal lattice
    a, b, c = lattice_vectors
    V = np.dot(a, np.cross(b, c))
    a_star = 2 * np.pi * np.cross(b, c) / V
    b_star = 2 * np.pi * np.cross(c, a) / V
    c_star = 2 * np.pi * np.cross(a, b) / V
    
    # Surface normal (reciprocal lattice direction)
    G = h * a_star + k * b_star + l * c_star
    normal = G / np.linalg.norm(G)
    
    # Find two in-plane vectors
    # Method: find lattice vectors perpendicular to normal
    # Start with a vector not parallel to normal
    if abs(normal[0]) < 0.9:
        arbitrary = np.array([1, 0, 0])
    else:
        arbitrary = np.array([0, 1, 0])
    
    # Gram-Schmidt to get orthogonal in-plane vector
    v1 = arbitrary - np.dot(arbitrary, normal) * normal
    v1 = v1 / np.linalg.norm(v1)
    
    # Second in-plane vector
    v2 = np.cross(normal, v1)
    v2 = v2 / np.linalg.norm(v2)
    
    return normal, v1, v2

# Test for Cu(111)
normal, v1, v2 = find_surface_vectors(1, 1, 1, cu_bulk.lattice_vectors)
print(f"Cu(111) surface:")
print(f"  Normal: {normal}")
print(f"  In-plane v1: {v1}")
print(f"  In-plane v2: {v2}")

## 7.3 Building Slabs for Common Surfaces

For FCC metals, the most important low-index surfaces are:
- **(111)**: Closest-packed, hexagonal surface
- **(100)**: Square surface
- **(110)**: Rectangular surface, more open

In [ ]:
class Slab:
    """A surface slab model."""
    
    def __init__(self, name: str, surface_vectors: np.ndarray, 
                 vacuum: float = 0.0):
        """Initialize slab.
        
        Args:
            name: Slab identifier
            surface_vectors: 3x3 array [a_surf, b_surf, c_surf]
                a_surf, b_surf: in-plane vectors
                c_surf: out-of-plane (normal direction)
            vacuum: Vacuum layer thickness in Å
        """
        self.name = name
        self.surface_vectors = np.array(surface_vectors, dtype=float)
        self.vacuum = vacuum
        self.atoms: List[Tuple[str, np.ndarray]] = []  # (symbol, cartesian_pos)
    
    def add_atom(self, symbol: str, cart_position: np.ndarray) -> None:
        """Add atom with Cartesian coordinates."""
        self.atoms.append((symbol, np.array(cart_position, dtype=float)))
    
    @property
    def n_atoms(self) -> int:
        return len(self.atoms)
    
    @property
    def thickness(self) -> float:
        """Slab thickness (excluding vacuum)."""
        if not self.atoms:
            return 0.0
        z_coords = [pos[2] for _, pos in self.atoms]
        return max(z_coords) - min(z_coords)
    
    @property
    def surface_area(self) -> float:
        """Surface area in Å²."""
        return np.linalg.norm(np.cross(self.surface_vectors[0], 
                                        self.surface_vectors[1]))
    
    def center_slab(self) -> None:
        """Center the slab in the z-direction."""
        if not self.atoms:
            return
        z_coords = [pos[2] for _, pos in self.atoms]
        z_center = (max(z_coords) + min(z_coords)) / 2
        cell_z = self.surface_vectors[2, 2]
        shift = cell_z / 2 - z_center
        
        self.atoms = [(sym, pos + np.array([0, 0, shift])) 
                      for sym, pos in self.atoms]
    
    def add_vacuum(self, vacuum_height: float) -> None:
        """Add or adjust vacuum layer."""
        self.vacuum = vacuum_height
        # Adjust c-vector to include vacuum
        self.surface_vectors[2, 2] = self.thickness + vacuum_height
        self.center_slab()

print("Slab class defined!")

In [ ]:
def build_fcc_111_slab(element: str, a: float, n_layers: int = 4,
                        vacuum: float = 15.0) -> Slab:
    """Build FCC(111) surface slab.
    
    The (111) surface has hexagonal symmetry with ABC stacking.
    
    Args:
        element: Element symbol
        a: Bulk lattice constant
        n_layers: Number of atomic layers
        vacuum: Vacuum thickness in Å
    """
    # Surface lattice vectors
    # For FCC(111): hexagonal surface cell
    # a_surf = a/sqrt(2) along [1,-1,0]
    # b_surf = a/sqrt(2) along [0,1,-1], 60° from a_surf
    
    a_surf = a / np.sqrt(2)
    
    # In-plane vectors (hexagonal)
    v1 = np.array([a_surf, 0, 0])
    v2 = np.array([a_surf * 0.5, a_surf * np.sqrt(3)/2, 0])
    
    # Interlayer spacing along [111]
    d_111 = a / np.sqrt(3)
    
    # c-vector: perpendicular (will be adjusted for vacuum)
    c_length = n_layers * d_111 + vacuum
    v3 = np.array([0, 0, c_length])
    
    slab = Slab(f"{element}(111)_{n_layers}L", np.array([v1, v2, v3]), vacuum)
    
    # ABC stacking positions
    # A: (0, 0), B: (1/3, 1/3), C: (2/3, 2/3) in fractional surface coords
    stacking = [
        np.array([0, 0]),           # A
        np.array([1/3, 1/3]),       # B  
        np.array([2/3, 2/3])        # C
    ]
    
    for layer in range(n_layers):
        stack_pos = stacking[layer % 3]
        z = layer * d_111 + vacuum / 2  # Start from vacuum/2
        
        # Convert to Cartesian
        xy = stack_pos[0] * v1 + stack_pos[1] * v2
        position = np.array([xy[0], xy[1], z])
        
        slab.add_atom(element, position)
    
    return slab

# Build Cu(111) slab
cu_111 = build_fcc_111_slab('Cu', 3.615, n_layers=6, vacuum=15.0)
print(f"Cu(111) slab: {cu_111.n_atoms} atoms")
print(f"Surface area: {cu_111.surface_area:.2f} Å²")
print(f"Slab thickness: {cu_111.thickness:.2f} Å")

In [ ]:
def build_fcc_100_slab(element: str, a: float, n_layers: int = 4,
                        vacuum: float = 15.0) -> Slab:
    """Build FCC(100) surface slab.
    
    The (100) surface has square symmetry.
    """
    # Surface lattice vectors
    # For FCC(100): square surface cell
    # a_surf = a/sqrt(2) along [1,-1,0]
    # b_surf = a/sqrt(2) along [0,0,1] (projected)
    
    a_surf = a / np.sqrt(2)
    
    v1 = np.array([a_surf, 0, 0])
    v2 = np.array([0, a_surf, 0])
    
    # Interlayer spacing along [100]
    d_100 = a / 2
    
    c_length = n_layers * d_100 + vacuum
    v3 = np.array([0, 0, c_length])
    
    slab = Slab(f"{element}(100)_{n_layers}L", np.array([v1, v2, v3]), vacuum)
    
    # Alternating layer positions
    for layer in range(n_layers):
        z = layer * d_100 + vacuum / 2
        
        if layer % 2 == 0:
            # A layer at (0, 0)
            position = np.array([0, 0, z])
        else:
            # B layer at (0.5, 0.5)
            position = np.array([a_surf/2, a_surf/2, z])
        
        slab.add_atom(element, position)
    
    return slab

# Build Cu(100) slab
cu_100 = build_fcc_100_slab('Cu', 3.615, n_layers=6, vacuum=15.0)
print(f"Cu(100) slab: {cu_100.n_atoms} atoms")
print(f"Surface area: {cu_100.surface_area:.2f} Å²")

In [ ]:
def build_fcc_110_slab(element: str, a: float, n_layers: int = 4,
                        vacuum: float = 15.0) -> Slab:
    """Build FCC(110) surface slab.
    
    The (110) surface has rectangular symmetry.
    More open than (111) or (100).
    """
    # Surface lattice vectors for (110)
    # Along [001]: length a
    # Along [1,-1,0]: length a*sqrt(2)
    
    v1 = np.array([a, 0, 0])                      # [001] direction
    v2 = np.array([0, a * np.sqrt(2), 0])         # [1-10] direction
    
    # Interlayer spacing along [110]
    d_110 = a / (2 * np.sqrt(2))
    
    c_length = n_layers * d_110 + vacuum
    v3 = np.array([0, 0, c_length])
    
    slab = Slab(f"{element}(110)_{n_layers}L", np.array([v1, v2, v3]), vacuum)
    
    # (110) has 2 atoms per surface unit cell in alternating rows
    for layer in range(n_layers):
        z = layer * d_110 + vacuum / 2
        
        if layer % 2 == 0:
            pos1 = np.array([0, 0, z])
            pos2 = np.array([a/2, a*np.sqrt(2)/2, z])
        else:
            pos1 = np.array([a/2, 0, z])
            pos2 = np.array([0, a*np.sqrt(2)/2, z])
        
        slab.add_atom(element, pos1)
        slab.add_atom(element, pos2)
    
    return slab

# Build Cu(110) slab
cu_110 = build_fcc_110_slab('Cu', 3.615, n_layers=6, vacuum=15.0)
print(f"Cu(110) slab: {cu_110.n_atoms} atoms")
print(f"Surface area: {cu_110.surface_area:.2f} Å²")

## 7.4 Visualizing Slabs

In [ ]:
def plot_slab(slab: Slab, repeat: Tuple[int, int] = (2, 2),
              view: str = 'side') -> None:
    """Visualize a slab model.
    
    Args:
        slab: Slab to visualize
        repeat: Surface cell repetitions for visualization
        view: 'side' or 'top'
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    v1, v2, v3 = slab.surface_vectors
    
    # Collect atoms with periodic images
    all_positions = []
    all_symbols = []
    
    for sym, pos in slab.atoms:
        for i in range(repeat[0]):
            for j in range(repeat[1]):
                new_pos = pos + i * v1 + j * v2
                all_positions.append(new_pos)
                all_symbols.append(sym)
    
    all_positions = np.array(all_positions)
    
    # Plot atoms
    unique_symbols = list(set(all_symbols))
    for elem in unique_symbols:
        mask = [s == elem for s in all_symbols]
        elem_pos = all_positions[mask]
        props = Crystal.ELEMENTS.get(elem, {'color': 'gray', 'radius': 1.0})
        ax.scatter(elem_pos[:, 0], elem_pos[:, 1], elem_pos[:, 2],
                   s=props['radius']*300, c=props['color'],
                   edgecolors='black', label=elem, alpha=0.9)
    
    # Draw cell boundaries
    cell_verts = [
        np.zeros(3), v1*repeat[0], v1*repeat[0]+v2*repeat[1], v2*repeat[1],
        v3, v1*repeat[0]+v3, v1*repeat[0]+v2*repeat[1]+v3, v2*repeat[1]+v3
    ]
    edges = [(0,1), (1,2), (2,3), (3,0),
             (4,5), (5,6), (6,7), (7,4),
             (0,4), (1,5), (2,6), (3,7)]
    
    for i, j in edges:
        ax.plot3D(*zip(cell_verts[i], cell_verts[j]), 'k--', alpha=0.3)
    
    # Draw vacuum region
    if slab.vacuum > 0:
        z_top = max(p[2] for _, p in slab.atoms) + 1
        z_cell = v3[2]
        ax.axhspan(z_top, z_cell, alpha=0.1, color='blue', label='Vacuum')
    
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.set_title(slab.name)
    ax.legend()
    
    if view == 'top':
        ax.view_init(elev=90, azim=0)
    else:
        ax.view_init(elev=20, azim=45)
    
    plt.tight_layout()
    return fig, ax

# Visualize the three low-index surfaces
slabs = [cu_111, cu_100, cu_110]

for slab in slabs:
    plot_slab(slab, repeat=(3, 3), view='side')
    plt.show()

## 7.5 General Surface Builder

For arbitrary Miller indices, we need a more general approach.

In [ ]:
def find_minimal_surface_cell(h: int, k: int, l: int, 
                               lattice_vectors: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Find minimal surface unit cell vectors for arbitrary (hkl).
    
    Returns:
        (in_plane_vectors, surface_normal)
        in_plane_vectors: 2x3 array of surface lattice vectors
        surface_normal: unit normal vector
    """
    a, b, c = lattice_vectors
    
    # Surface normal from reciprocal lattice
    V = np.dot(a, np.cross(b, c))
    a_star = 2 * np.pi * np.cross(b, c) / V
    b_star = 2 * np.pi * np.cross(c, a) / V
    c_star = 2 * np.pi * np.cross(a, b) / V
    
    G = h * a_star + k * b_star + l * c_star
    normal = G / np.linalg.norm(G)
    
    # Find in-plane lattice vectors
    # Search for short vectors perpendicular to normal
    candidates = []
    
    search_range = max(abs(h), abs(k), abs(l), 3)
    for i in range(-search_range, search_range + 1):
        for j in range(-search_range, search_range + 1):
            for m in range(-search_range, search_range + 1):  # renamed k to m
                if i == j == m == 0:
                    continue
                vec = i * a + j * b + m * c
                # Check if perpendicular to normal
                if abs(np.dot(vec, normal)) < 1e-6:
                    length = np.linalg.norm(vec)
                    candidates.append((length, vec))
    
    # Sort by length
    candidates.sort(key=lambda x: x[0])
    
    if len(candidates) < 2:
        raise ValueError(f"Could not find in-plane vectors for ({h}{k}{l})")
    
    # Select shortest vector as v1
    v1 = candidates[0][1]
    
    # Find shortest v2 that's not parallel to v1
    for _, v2 in candidates[1:]:
        # Check not parallel
        cross = np.cross(v1, v2)
        if np.linalg.norm(cross) > 1e-6:
            break
    
    return np.array([v1, v2]), normal

# Test for high-index surface
in_plane, normal = find_minimal_surface_cell(2, 1, 1, cu_bulk.lattice_vectors)
print(f"Cu(211) surface:")
print(f"  In-plane v1: {in_plane[0]} (length: {np.linalg.norm(in_plane[0]):.2f} Å)")
print(f"  In-plane v2: {in_plane[1]} (length: {np.linalg.norm(in_plane[1]):.2f} Å)")
print(f"  Normal: {normal}")

## 7.6 Surface Terminations

For compound materials, different surface terminations are possible.

Example: SrTiO₃(100) can be:
- **SrO-terminated**: SrO layer exposed
- **TiO₂-terminated**: TiO₂ layer exposed

In [ ]:
def build_perovskite_100_slab(A: str, B: str, X: str, a: float,
                               n_unit_cells: int = 3,
                               termination: str = 'AO',
                               vacuum: float = 15.0) -> Slab:
    """Build perovskite ABX3 (100) surface.
    
    Args:
        termination: 'AO' or 'BO2'
    """
    v1 = np.array([a, 0, 0])
    v2 = np.array([0, a, 0])
    
    # Each unit cell has 2 layers: AO and BO2
    n_layers = 2 * n_unit_cells
    d_layer = a / 2  # Layer spacing
    
    c_length = n_layers * d_layer + vacuum
    v3 = np.array([0, 0, c_length])
    
    slab = Slab(f"{A}{B}{X}3(100)_{termination}", np.array([v1, v2, v3]), vacuum)
    
    for i in range(n_layers):
        z = i * d_layer + vacuum / 2
        
        if termination == 'AO':
            if i % 2 == 0:  # AO layer
                slab.add_atom(A, np.array([0, 0, z]))
                slab.add_atom(X, np.array([a/2, a/2, z]))
            else:  # BO2 layer
                slab.add_atom(B, np.array([a/2, a/2, z]))
                slab.add_atom(X, np.array([a/2, 0, z]))
                slab.add_atom(X, np.array([0, a/2, z]))
        else:  # BO2 termination
            if i % 2 == 0:  # BO2 layer
                slab.add_atom(B, np.array([0, 0, z]))
                slab.add_atom(X, np.array([a/2, 0, z]))
                slab.add_atom(X, np.array([0, a/2, z]))
            else:  # AO layer
                slab.add_atom(A, np.array([a/2, a/2, z]))
                slab.add_atom(X, np.array([0, 0, z]))
    
    return slab

# Build SrTiO3 with different terminations
sro_terminated = build_perovskite_100_slab('Sr', 'Ti', 'O', 3.905, 
                                            n_unit_cells=3, termination='AO')
tio2_terminated = build_perovskite_100_slab('Sr', 'Ti', 'O', 3.905,
                                             n_unit_cells=3, termination='BO2')

print(f"SrO-terminated: {sro_terminated.n_atoms} atoms")
print(f"TiO2-terminated: {tio2_terminated.n_atoms} atoms")

## 7.7 Adding Adsorbates

To model adsorption, we place molecules at specific surface sites.

In [ ]:
def get_surface_sites_fcc_111(slab: Slab) -> dict:
    """Get adsorption sites for FCC(111) surface.
    
    Returns positions for:
    - 'top': directly above surface atom
    - 'bridge': between two surface atoms
    - 'hcp': hollow site (above 2nd layer atom)
    - 'fcc': hollow site (no atom below)
    """
    v1, v2 = slab.surface_vectors[:2]
    
    # Find topmost atom
    z_max = max(pos[2] for _, pos in slab.atoms)
    top_atom = None
    for sym, pos in slab.atoms:
        if abs(pos[2] - z_max) < 0.01:
            top_atom = pos.copy()
            break
    
    sites = {
        'top': top_atom,
        'bridge': top_atom + v1/2,
        'hcp': top_atom + (v1 + v2)/3,
        'fcc': top_atom + 2*(v1 + v2)/3,
    }
    
    return sites

def add_adsorbate(slab: Slab, adsorbate_atoms: List[Tuple[str, np.ndarray]],
                  site: str, height: float, sites_dict: dict = None) -> Slab:
    """Add adsorbate molecule to slab.
    
    Args:
        slab: Surface slab
        adsorbate_atoms: List of (symbol, relative_position) for adsorbate
        site: Adsorption site ('top', 'bridge', 'hcp', 'fcc')
        height: Height above surface in Å
        sites_dict: Dictionary of site positions
    """
    if sites_dict is None:
        sites_dict = get_surface_sites_fcc_111(slab)
    
    base_pos = sites_dict[site].copy()
    z_surface = max(pos[2] for _, pos in slab.atoms)
    
    for symbol, rel_pos in adsorbate_atoms:
        ads_pos = np.array([
            base_pos[0] + rel_pos[0],
            base_pos[1] + rel_pos[1],
            z_surface + height + rel_pos[2]
        ])
        slab.add_atom(symbol, ads_pos)
    
    return slab

# Example: Add CO molecule to Cu(111)
cu_111_clean = build_fcc_111_slab('Cu', 3.615, n_layers=4, vacuum=15.0)

# CO molecule: C at bottom, O at top
co_molecule = [
    ('C', np.array([0, 0, 0])),
    ('O', np.array([0, 0, 1.13]))  # C-O bond length ~1.13 Å
]

sites = get_surface_sites_fcc_111(cu_111_clean)
cu_111_co = add_adsorbate(cu_111_clean, co_molecule, 'top', height=1.85, sites_dict=sites)

print(f"Cu(111) + CO: {cu_111_co.n_atoms} atoms")

# Visualize
plot_slab(cu_111_co, repeat=(2, 2))
plt.show()

---

## Practice Exercises

### Exercise 7.1: BCC(110) Surface
Build a BCC(110) slab for iron (a = 2.87 Å).

In [ ]:
# YOUR CODE HERE
def build_bcc_110_slab(element: str, a: float, n_layers: int = 4,
                        vacuum: float = 15.0) -> Slab:
    """Build BCC(110) surface slab.
    
    The (110) is the most densely packed BCC surface.
    """
    # TODO: Implement
    pass

### Exercise 7.2: Surface Energy

Surface energy is calculated as:
$$\gamma = \frac{E_{slab} - N \cdot E_{bulk}}{2A}$$

Write a function to prepare slab calculations for surface energy.

In [ ]:
# YOUR CODE HERE
def calculate_surface_energy(E_slab: float, N_atoms: int,
                              E_bulk_per_atom: float,
                              surface_area: float) -> float:
    """Calculate surface energy.
    
    Args:
        E_slab: Total energy of slab (eV)
        N_atoms: Number of atoms in slab
        E_bulk_per_atom: Bulk energy per atom (eV)
        surface_area: Surface area in Å²
    
    Returns:
        Surface energy in J/m² (divide by 2 for two surfaces)
    """
    # TODO: Implement
    pass

### Exercise 7.3: Stepped Surface

High-index surfaces like (211) contain steps. Build a Cu(211) surface.

In [ ]:
# YOUR CODE HERE
# Hint: Use find_minimal_surface_cell() and build atoms layer by layer

---

## Key Takeaways

1. **Slab models** represent surfaces as periodic 2D systems with vacuum
2. **Surface vectors** are found using crystallography (perpendicular to [hkl])
3. **Low-index surfaces** (111), (100), (110) are most stable
4. **Termination** matters for compound materials
5. **Adsorbates** are placed at specific surface sites

## Next Chapter Preview

In Chapter 8, we'll work with **molecules** - building, manipulating, and analyzing molecular structures.